# Implementation of Target Entropy Encoding (Developed Algorithm)

In [1]:
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
import math

In [4]:
df = pd.read_csv(r"C:\Users\John Taiwo\Downloads\toyota.csv")


In [5]:
df['price'] = df['price'].apply(lambda x: 1 if x > df.price.mean() else 0)

df_1 = df.iloc[:5000,:]
df_2 = df.iloc[5001:,:]
df = df_1

df_2.info()
df_2.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1737 entries, 5001 to 6737
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         1737 non-null   object 
 1   year          1737 non-null   int64  
 2   price         1737 non-null   int64  
 3   transmission  1737 non-null   object 
 4   mileage       1737 non-null   int64  
 5   fuelType      1737 non-null   object 
 6   tax           1737 non-null   int64  
 7   mpg           1737 non-null   float64
 8   engineSize    1737 non-null   float64
dtypes: float64(2), int64(4), object(3)
memory usage: 122.3+ KB


,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
5001,Aygo,2019,0,Manual,4535,Petrol,145,57.7,1.0
5002,Aygo,2019,0,Manual,66,Petrol,145,57.7,1.0
5003,Aygo,2020,0,Manual,4578,Petrol,150,56.5,1.0
5004,Aygo,2016,0,Manual,29906,Petrol,0,69.0,1.0
5005,Aygo,2020,0,Semi-Auto,4568,Petrol,145,56.5,1.0


In [6]:
def entropy(df, col):
    t0_prob = len(df[df[col] == 0])/len(df)
    t1_prob = len(df[df[col] == 1])/len(df)
    if t1_prob == 0 or t0_prob == 0:
        total_ent = 0
    else:
        t0_ent = (-t0_prob)*(math.log2(t0_prob))
        t1_ent = (-t1_prob)*(math.log2(t1_prob))
        total_ent = t0_ent + t1_ent
    return total_ent

In [7]:
def cont(df, column, target):
    
    #temp_df =
    
    target_ent = entropy(df, target)
    cat_list = df[column].unique()
    #print(cat_list)
    test_list=[]
    
    enc_col = column+'_enc'
    df.loc[:,enc_col] = df[column]

    for cat in cat_list:
        
        df_cat = df[df[column] == cat]
        ent_cat = entropy(df_cat, target)
        prob_cat = len(df_cat)/len(df)
        cont_cat = (prob_cat*ent_cat)/target_ent
        
        
        df[enc_col] = df[enc_col].replace([cat],cont_cat)
        
        #test_list.append(cont_cat)
        #print(cont_cat)
        
    return df

In [8]:
cont(df, 'transmission', 'price')
cont(df, 'fuelType', 'price')
cont(df, 'model', 'price')

df.head()

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,transmission_enc,fuelType_enc,model_enc
0,GT86,2016,1,Manual,24089,Petrol,265,36.2,2.0,0.355871,0.38797,0.006801
1,GT86,2017,1,Manual,18615,Petrol,145,36.2,2.0,0.355871,0.38797,0.006801
2,GT86,2015,1,Manual,27469,Petrol,265,36.2,2.0,0.355871,0.38797,0.006801
3,GT86,2017,1,Manual,14736,Petrol,150,36.2,2.0,0.355871,0.38797,0.006801
4,GT86,2017,1,Manual,36284,Petrol,145,36.2,2.0,0.355871,0.38797,0.006801


In [9]:
print(df.transmission.unique())
print(df.fuelType.unique())
print(df.model.unique())

['Manual' 'Automatic' 'Semi-Auto' 'Other']
['Petrol' 'Other' 'Hybrid' 'Diesel']
[' GT86' ' Corolla' ' RAV4' ' Yaris' ' Auris' ' Aygo']


In [10]:

def rep(column):
    to_rep = df_2[column].unique()
#print('to_rep='+str(to_rep))
    for i in to_rep:
        if i not in df[column].unique():
            df_2[column] = df_2[column].replace([i],0)
        else:
            rep_val = df[df[column]==i][column+'_enc'].values
            rep_val = rep_val[0]
#print('rep_val='+str(rep_val))
            df_2[column] = df_2[column].replace([i],rep_val)
#print(df_2.transmission)

    return df[column].unique(),df_2[column].unique()

col_list = ['transmission', 'fuelType', 'model']
for col in col_list:
    print(rep(col))
    #print(df[col].unique())
    print()

(array(['Manual', 'Automatic', 'Semi-Auto', 'Other'], dtype=object), array([0.355871 , 0.0147314, 0.3912999]))

(array(['Petrol', 'Other', 'Hybrid', 'Diesel'], dtype=object), array([0.3879698 , 0.00410075, 0.30035821, 0.05393965]))

(array([' GT86', ' Corolla', ' RAV4', ' Yaris', ' Auris', ' Aygo'],
      dtype=object), array([0.00894957, 0.        ]))



In [11]:
df_2.head()

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
5001,0.00895,2019,0,0.355871,4535,0.38797,145,57.7,1.0
5002,0.00895,2019,0,0.355871,66,0.38797,145,57.7,1.0
5003,0.00895,2020,0,0.355871,4578,0.38797,150,56.5,1.0
5004,0.00895,2016,0,0.355871,29906,0.38797,0,69.0,1.0
5005,0.00895,2020,0,0.014731,4568,0.38797,145,56.5,1.0


In [91]:
train_features = df[['year', 'mileage', 'tax',
       'mpg', 'engineSize','transmission_enc','model_enc','fuelType_enc']]
train_labels = df['price']

test_features = df_2[['year', 'mileage', 'tax',
       'mpg', 'engineSize','transmission','model','fuelType']]
test_labels = df_2['price']

In [139]:
scalar = StandardScaler()
train_features = scalar.fit_transform(train_features)
test_features = scalar.transform(test_features)

In [178]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators = 100)   
model.fit(train_features,train_labels)

RandomForestClassifier()

In [179]:
print(model.score(train_features, train_labels))

0.9994


In [180]:
print(model.score(test_features, test_labels))

0.8981001727115717


In [174]:
from sklearn.metrics import confusion_matrix
confusion_matrix(test_labels, y_predict)

array([[704,  99],
       [ 60, 874]], dtype=int64)